In [1]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [2]:
RAVDESS_DIR = "./archive2/"

In [3]:
file_paths = []
for root, dirs, files in os.walk(RAVDESS_DIR):
    for file in files:
        if file.endswith(".wav"):
            file_paths.append(os.path.join(root, file))

In [8]:
def extract_features(file_path, max_pad_len=400):
    audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    if (max_pad_len > mfccs.shape[1]):
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfccs = mfccs[:, :max_pad_len]
    
    return mfccs


In [9]:
def extract_labels(file_path):
    filename = os.path.basename(file_path)
    parts = filename.split("-")
    emotion = int(parts[2])
    
    return emotion

In [10]:
features = []
labels = []
for path in file_paths:
    features.append(extract_features(path))
    labels.append(extract_labels(path))

In [11]:
features = np.array(features)
labels = np.array(labels)

In [12]:
features = features.reshape(features.shape[0], features.shape[1], features.shape[2], 1)

In [13]:
labels = to_categorical(labels)

In [14]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [15]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(features.shape[1], features.shape[2], 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(labels.shape[1], activation='softmax'))

In [16]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [17]:
model.fit(features_train, labels_train, batch_size=32, epochs=50, verbose=1, validation_data=(features_test, labels_test))

Epoch 1/50


72/72 [==============================] - 13s 166ms/step - loss: 25.4189 - accuracy: 0.1328 - val_loss: 2.1889 - val_accuracy: 0.1059
Epoch 2/50
72/72 [==============================] - 11s 156ms/step - loss: 2.1818 - accuracy: 0.1293 - val_loss: 2.1782 - val_accuracy: 0.1372
Epoch 3/50
72/72 [==============================] - 11s 148ms/step - loss: 2.1716 - accuracy: 0.1393 - val_loss: 2.1482 - val_accuracy: 0.1198
Epoch 4/50
72/72 [==============================] - 11s 149ms/step - loss: 2.1550 - accuracy: 0.1471 - val_loss: 2.1489 - val_accuracy: 0.1163
Epoch 5/50
72/72 [==============================] - 11s 150ms/step - loss: 2.1486 - accuracy: 0.1432 - val_loss: 2.1091 - val_accuracy: 0.1337
Epoch 6/50
72/72 [==============================] - 11s 148ms/step - loss: 2.1407 - accuracy: 0.1484 - val_loss: 2.1371 - val_accuracy: 0.1146
Epoch 7/50
72/72 [==============================] - 11s 148ms/step - loss: 2.1315 - accuracy: 0.1450 - val_loss: 2.1413 - val_accuracy: 0.1

In [18]:
loss, accuracy = model.evaluate(features_test, labels_test, verbose=0)

In [19]:
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 10.59%
